# SQL-3. Соединение таблиц

## 1. Знакомимся с данными

✍ Вы уже умеете делать запросы к одной таблице, использовать агрегатные функции и фильтровать данные в выводе. Но в реальных условиях базы данных обычно содержат множество таблиц и при запросе необходимо обращаться к нескольким таблицам. Освоением этого навыка мы сейчас и займёмся ↓

→ Первым делом — знакомимся с датасетом.

В этом модуле мы будем работать с таблицами о футбольных матчах и командах.

Таблицы этого модуля, как и все другие в курсе, лежат в схеме sql в Metabase. Нам понадобятся таблицы teams и matches.


Таблица teams с данными о командах
Название столбца
Содержимое столбца
id	id команды
api_id	ключ на таблицу matches
long_name	полное название команды
short_name	сокращённое название команды


Таблица matches с данными о матчах
Название столбца
Содержимое столбца
id	id матча
season	сезон
date	дата матча
home_team_api_id	api_id домашней команды, ключ на таблицу teams
away_team_api_id	api_id гостевой команды, ключ на таблицу teams
home_team_goals	количество голов домашней команды
away_team_goals	количество голов гостевой команды

Примечание. Как и в предыдущих модулях, вы можете посмотреть подробный разбор структуры предлагаемых запросов во вкладке Детализация.

### Задание 1.1
5 points possible (graded)
Исследуйте данные датасета самостоятельно.

→ Сколько различных полных названий команд в таблице teams?

SELECT COUNT(DISTINCT long_name) AS unique_team_names
FROM sql.teams;

Ответ: 296

→ Сколько в таблице teams команд с коротким названием VAL?
В ответе укажите только число.

SELECT DISTINCT
    count(short_name) AS short_name
FROM 
    sql.teams
WHERE short_name = 'VAL'    

Ответ: 3

→ Информацию о скольких матчах содержит таблица matches?
В ответе укажите только число.

SELECT DISTINCT
    count(id)
FROM 
    sql.matches

Ответ: 25083

Данные за какие сезоны даны в таблице matches?
Ответ введите в формате 2019/2020.

SELECT 
    MIN(season) AS min_season, 
    MAX(season) AS max_season
FROM sql.matches;


Ответ:
2008/2009
2015/2016


### Задание 1.2 (External resource)

Напишите запрос, который выведет сезон (season), а также общее количество забитых мячей домашними (total_home_goals) и гостевыми (total_away_goals) командами.

Отсортируйте по столбцу с сезоном в порядке возрастания.


SELECT 
    season, 
    SUM(home_team_goals) AS total_home_goals,
    SUM(away_team_goals) AS total_away_goals
FROM sql.matches
GROUP BY season
ORDER BY season ASC



## 2. Соединение таблиц по ключу

#### Объединяем таблицы без операторов

✍ Существует несколько способов соединения таблиц. Мы познакомимся со всеми основными операторами, которые используются для этих нужд, но начнём с простого метода объединения таблиц — без операторов.


Чтобы соединить две таблицы между собой, достаточно записать названия таблиц через запятую в разделе from. Что произойдёт в таком случае?

SELECT * /*выбор всех полей*/
FROM
    sql.teams, /*таблица с командами*/
    sql.matches /*таблица с матчами*/

Каждая запись, которая есть в таблице teams, будет соединена с каждой записью в таблице matches.

Это действие также называют **декартовым произведением таблиц.**

### Задание 2.1 (External resource)

Напишите запрос, который выведет количество строк соединённой таблицы.

SELECT 
    count(*)
FROM
    sql.teams,
    sql.matches



В данном случае соединение таблиц не даёт практической пользы: мы получили очень много записей, которые никак не можем интерпретировать, потому что команды не соответствуют матчам.

Давайте исправим это. В таблице teams есть столбец api_id, а таблица matches содержит столбцы home_team_api_id и away_team_api_id — это ключи таблиц, по которым они соединяются.

**Ключ — это поле (столбец) в таблице, которое позволяет однозначно идентифицировать запись (строку).**

Чтобы соединить таблицы и получить данные о домашней команде по каждому матчу, добавим условие
where home_team_api_id = api_id.

SELECT * /*выбор всех полей в таблице*/
FROM
    sql.teams, /*таблица с командами*/
    sql.matches /*таблица с матчами*/
WHERE home_team_api_id = api_id /*условие: home_team_api_id таблицы matches равен api_id таблицы teams*/

Аналогично можем получить данные о гостевых командах: необходимо изменить условие на
where away_team_api_id = api_id.

SELECT * /*выбор всех полей в таблицы*/
FROM
    sql.teams, /*таблица с командами*/
    sql.matches /*таблица с матчами*/
WHERE away_team_api_id = api_id /*условие: away_team_api_id таблицы matches равен api_id таблицы teams*/

Итак, мы только что объединили таблицы по **ключу**.

Вы уже знакомы с ключами по таблице pokemon (там в этой роли выступал столбец id). Ключи нужны для того, чтобы иметь возможность не перепутать между собой различные записи.

→ Например, у нас есть несколько команд с одинаковым названием: Polonia Bytom, Widzew Łódź и Royal Excel Mouscron — хотя это разные команды, с разными id.
Кроме того, как мы уже смогли убедиться, ключи используются для соединения таблиц между собой.

Ключи бывают двух основных типов:

**Primary** — первичный ключ — служит для идентификации текущей таблицы и, как правило, идёт первым в списке столбцов. Всегда уникален: повторяющихся значений в основной таблице быть не может.

**Foreign** — внешний ключ — представляет собой ссылку на другую таблицу.

Как правило, названия ключей имеют «хвост», который позволяет их идентифицировать: например, _id, _rk, _cd, _pk (от primary_key), _fk (от foreign_key) и другие.

Обратите внимание! В данном датасете ключ api_id таблицы teams может быть использован в разных значениях. Его можно использовать для того, чтобы получить информацию о домашней (home) или гостевой (away) команде.

Вы могли заметить, что в последних двух запросах получилось очень много столбцов. Как и при работе с одиночной таблицей, мы можем выбирать, какие столбцы соединённой таблицы выводить.

С помощью известного нам запроса получим названия команд, игравших домашние матчи, и счёт матчей.

SELECT 
    long_name, /*столбец long_name таблицы teams*/
    home_team_goals, /*столбец home_team_goals таблицы matches*/
    away_team_goals /*столбец away_team_goals таблицы matches*/
FROM
    sql.teams, /*таблица с командами*/
    sql.matches /*таблица с матчами*/
WHERE home_team_api_id = api_id /*условие: home_team_api_id таблицы matches равен api_id таблицы teams*/

### Задание 2.2 (External resource)

Напишите запрос, который выведет таблицу с результатами матчей для гостевых команд, содержащую:

названия гостевых команд (long_name),
количество забитых мячей домашней команды (home_team_goals),
количество забитых мячей гостевой команды (away_team_goals).

SELECT 
    long_name, /*столбец long_name таблицы teams*/
    home_team_goals, /*столбец home_team_goals таблицы matches*/
    away_team_goals /*столбец away_team_goals таблицы matches*/
FROM
    sql.teams, /*таблица с командами*/
    sql.matches /*таблица с матчами*/
WHERE away_team_api_id = api_id /*условие: away_team_api_id таблицы matches равен api_id таблицы teams*/


## 3. Знакомимся с JOIN

В прошлом юните для соединения таблиц мы использовали условие в разделе where, чтобы показать принцип работы оператора JOIN.

JOIN — это оператор SQL, который позволяет соединять таблицы по условию.

В качестве примера используем запрос из предыдущего юнита.

SELECT 
    long_name,  /*столбец long_name таблицы teams*/
    home_team_goals, /*столбец home_team_goals таблицы matches*/
    away_team_goals /*столбец away_team_goals таблицы matches*/
FROM
    sql.teams, /*таблица с командами*/
    sql.matches /*таблица с матчами*/
WHERE home_team_api_id = api_id /*условие: home_team_api_id таблицы matches равен api_id таблицы teams*/

и запишем его с использованием JOIN.


SELECT 
    long_name, /*столбец long_name таблицы teams*/
    home_team_goals, /*столбец home_team_goals таблицы matches*/
    away_team_goals /*столбец away_team_goals таблицы matches*/
FROM    
    sql.teams /*таблица с командами*/
JOIN sql.matches on home_team_api_id = api_id /*оператор соединения таблиц; таблица matches; условие: home_team_api_id таблицы matches равен api_id таблицы teams*/

### Задание 3.1 (External resource)
Выполните данный запрос:

SELECT *
FROM
    sql.teams,
    sql.matches
WHERE away_team_api_id = api_id

с использованием оператора JOIN, исключив оператор WHERE.


SELECT *
FROM
    sql.teams
JOIN sql.matches ON away_team_api_id = api_id


### Синтаксис

Оператор JOIN упрощает процесс соединения таблиц.

Его синтаксис можно представить следующим образом:

SELECT
        столбец1,
	столбец2,
	...
FROM
	таблица1
JOIN таблица2 ON условие

Порядок присоединения таблиц в данном случае не важен — результат будет одинаковым.

С помощью JOIN можно соединить и более двух таблиц.

SELECT
        столбец1,
	столбец2,
	...
FROM
	таблица1
JOIN таблица2 ON условие
JOIN таблица3 ON условие

В таблицах, которые мы соединяем, могут быть одинаковые названия столбцов.

→ К примеру, столбец id есть и в таблице matches, и в таблице teams. Такой запрос не будет обработан.

SELECT
    id /*столбец id, но не указано какой таблицы*/
FROM 
sql.teams /*таблица teams*/
    JOIN sql.matches ON home_team_api_id = api_id /*оператор соединения таблиц; таблица matches; условие: home_team_api_id таблицы matches равен api_id таблицы teams*/

В результате должно появиться сообщение об ошибке вроде такого: "... column id is ambiguous ...".

?
Что же делать в таком случае?

Можно указать, откуда мы хотим запросить данные, записав название таблицы перед столбцом через точку.

SELECT
    teams.id /*столбец id таблицы teams*/
FROM 
    sql.teams /*таблица teams*/
JOIN sql.matches ON home_team_api_id = api_id /*оператор соединения таблиц; таблица matches; условие: home_team_api_id таблицы matches равен api_id таблицы teams*/

Можно также выбрать и столбец из таблицы matches.

SELECT
    matches.id /*столбец id таблицы matches*/
FROM 
    sql.teams /*таблица teams*/
JOIN sql.matches ON home_team_api_id = api_id /*оператор соединения таблиц; таблица matches; условие: home_team_api_id таблицы matches равен api_id таблицы teams*/

Зачастую названия таблиц слишком длинные, так что использовать их неудобно.

Упростить обращение к различным таблицам можно, присвоив им сокращённые названия — алиасы (от англ. alias).

Синтаксис для указания алиаса такой же, как и для названия столбца.

SELECT
        столбец1,
	столбец2,
	...
FROM
	таблица1 AS короткое_название_1
JOIN таблица2 AS короткое_название_2 ON условие

Название записывается без пробелов и операторов.

Если необходимо записать название на русском языке ("таблица1") или с пробелом ("table 2"), то алиас можно обернуть в кавычки.

Важно! Обращаться по такому алиасу придётся также с помощью кавычек.


SELECT
	"таблица1".столбец1,
	"table 2".столбец2,
	...
FROM
	таблица1 AS "таблица1"
	JOIN таблица2 AS "table 2" ON условие


Использование таких алиасов считается плохой практикой как минимум по двум причинам:

обращаться с такими алиасами неудобно;
названия на кириллице можно задать не во всех СУБД.
Вместо алиасов с кавычками рекомендуется давать простое короткое название на латинице, без специальных символов и пробелов. Если таблиц немного и все названия начинаются с разных букв, можно присваивать алиасы по первой букве.

Ключевое слово as, как и в названии столбца, можно опустить в большинстве СУБД.

SELECT
        столбец1 новое_название_столбца,
	столбец2 новое_название_столбца,
	...
FROM
	таблица1 короткое_название_1
JOIN таблица2 короткое_название_2 ON условие

Давайте попробуем проделать то же самое для наших таблиц.



### Задание 3.2 (External resource)

Напишите запрос, который выведет два столбца: id матча (match_id) и id домашней команды (team_id). Отсортируйте по id матча в порядке возрастания значений.

SELECT
    m.id AS match_id,
    t.id AS team_id
FROM
    sql.matches m
JOIN
    sql.teams t ON m.home_team_api_id = t.api_id
ORDER BY
    m.id ASC;

Кажется, теперь мы можем соединить таблицы между собой и создать своё табло со счётом матчей, как на спортивных сайтах!


Давайте с помощью запроса SQL получим таблицу, содержащую:

название домашней команды;
количество забитых домашней командой голов;
количество забитых гостевой командой голов;
название гостевой команды.

SELECT
    h.long_name "домашняя команда", /*столбец long_name таблицы h*/
    m.home_team_goals "голы домашней команды", /*столбец home_team_goals таблицы m*/
    m.away_team_goals "голы гостевой команды", /*столбец away_team_goals таблицы m*/
    a.long_name "гостевая команда" /*столбец long_name таблицы a*/
FROM
    sql.matches m /*таблица matches с алиасом m*/
    JOIN sql.teams h ON m.home_team_api_id = h.api_id /*оператор соединения таблиц; таблица teams с алиасом h; условие: home_team_api_id таблицы m равен api_id таблицы h*/
    JOIN sql.teams a ON m.away_team_api_id = a.api_id /*оператор соединения таблиц; таблица teams с алиасом a; условие: away_team_api_id таблицы m равен api_id таблицы a*/

✍ А теперь попробуйте сделать это самостоятельно ↓

### Задание 3.3 (External resource)

Напишите запрос, который выведет столбцы:

id матча,
короткое название домашней команды (home_short),
короткое название гостевой команды (away_short).
Отсортируйте запрос по возрастанию id матча.


SELECT
    m.id,
    h.short_name home_short,
    a.short_name away_short
FROM
    sql.matches m /*таблица matches с алиасом m*/
    JOIN sql.teams h ON m.home_team_api_id = h.api_id /*оператор соединения таблиц; таблица teams с алиасом h; условие: home_team_api_id таблицы m равен api_id таблицы h*/
    JOIN sql.teams a ON m.away_team_api_id = a.api_id /*оператор соединения таблиц; таблица teams с алиасом a; условие: away_team_api_id таблицы m равен api_id таблицы a*/
ORDER BY 
    m.id ASC



## 4. Фильтрация и агрегатные функции

### Работа с объединёнными таблицами

✍ Соединять таблицы мы научились, теперь давайте научимся получать необходимые данные из нескольких таблиц ↓

Принцип построения запроса и порядок операторов такой же, как и с обычной таблицей.

Вспомним его:

SELECT... 
FROM... 
WHERE... 
GROUP BY... 
ORDER BY... 
LIMIT...

### Фильтрация данных

К соединённым таблицам применимы функции фильтрации данных.

Например, можно вывести id матчей, в которых команда Arsenal была гостевой.

SELECT 
    m.id /*столбец id таблицы m*/
FROM
    sql.teams t /*таблица teams с алиасом t*/
    JOIN sql.matches m ON m.away_team_api_id = t.api_id /*оператор соединения таблиц; таблица matches с алиасом m; условие: away_team_api_id таблицы m равен api_id таблицы t*/
WHERE long_name = 'Arsenal' /*long_name таблицы teams имеет значение Arsenal*/

Принципиальное отличие фильтрации данных по соединённым таблицам от аналогичного действия по одиночным таблицам заключается в том, что, фильтруя записи одной таблицы, мы также будем фильтровать и записи другой таблицы, поскольку соединённые на уровне запроса таблицы по сути являются единой таблицей.

Например, результат запроса

SELECT 
	m.id id_1,
	m.season,
	t.id id_2,
	t.long_name
FROM
	sql.teams t
JOIN sql.matches m ON m.away_team_api_id = t.api_id

можно разделить на две разные части:
Одна часть — таблица matches с алиасом m, вторая — teams с алиасом t, но после соединения они являются одной таблицей.

Таким образом, если вы отфильтруете данные по одной части таблицы, то другая, соединённая, часть пропадёт вместе с ней.

### Задание 4.1 (External resource)

Напишите запрос, который выведет полное название домашней команды (long_name), количество голов домашней команды (home_goal) и количество голов гостевой команды (away_goal) в матчах, где домашней командой были команды с коротким названием 'GEN'. Отсортируйте запрос по id матча в порядке возрастания.

SELECT 
    t.long_name AS long_name,
    m.home_team_goals AS home_goal, 
    m.away_team_goals AS away_goal     
FROM
    sql.teams t /*таблица teams с алиасом t*/
    JOIN sql.matches m ON m.home_team_api_id = t.api_id /*оператор соединения таблиц; таблица matches с алиасом m; условие: home_team_api_id таблицы m равен api_id таблицы t*/
WHERE t.short_name = 'GEN' /*Short_name таблицы teams имеет значение GEN*/
ORDER BY m.id ASC


Также мы можем отфильтровать записи сразу по двум таблицам.

Например, можно оставить только записи, в которых короткое название домашней команды GEN и матчи сезона 2008/2009.

SELECT *
FROM    
    sql.matches m
    JOIN sql.teams t on t.api_id = m.home_team_api_id
WHERE
    t.short_name = 'GEN'
    AND m.season = '2008/2009'

### Задание 4.2 (External resource)

Напишите запрос, чтобы вывести id матчей, короткое название домашней команды (home_short), короткое название гостевой команды (away_short) для матчей сезона 2011/2012, в которых участвовала команда с названием Liverpool. Отсортируйте по id матча в порядке возрастания.

SELECT 
    m.id,
    h.short_name AS home_short,
    a.short_name AS away_short
FROM
    sql.matches m
    JOIN sql.teams h ON m.home_team_api_id = h.api_id
    JOIN sql.teams a ON m.away_team_api_id = a.api_id    
WHERE (m.season = '2011/2012') AND (h.long_name = 'Liverpool' OR a.long_name = 'Liverpool') 
ORDER BY m.id ASC


SELECT
    m.id AS match_id,
    h.short_name AS home_short,
    a.short_name AS away_short
FROM
    sql.matches m
JOIN
    sql.teams h ON m.home_team_api_id = h.api_id
JOIN
    sql.teams a ON m.away_team_api_id = a.api_id
WHERE
    m.season = '2011/2012'
    AND (h.long_name = 'Liverpool' OR a.long_name = 'Liverpool')
ORDER BY
    m.id ASC;

### Агрегация данных

К соединённым таблицам также применимы любые агрегатные функции — самые важные функции для анализа данных.

Например, мы можем вывести сумму голов матча, забитых командами, агрегированную по гостевым командам (совокупное количество голов в матче, забитых обеими командами, суммированное в разрезе гостевых команд).

SELECT
    t.long_name, /*столбец long_name таблицы t*/
    SUM(m.home_team_goals) + SUM(m.away_team_goals) match_goals /*функция суммирования; столбец home_team_goals таблицы m; функция суммирования; столбец away_team_goals таблицы m; новое название столбца*/
FROM
    sql.matches m /*таблица matches с алиасом m*/
    JOIN sql.teams t ON m.away_team_api_id = t.api_id /*оператор соединения таблиц; таблица teams с алиасом t; условие: away_team_api_id таблицы m равен api_id таблицы t*/
GROUP BY t.id /*группировка по столбцу id таблицы t*/

Обратите внимание! В данном запросе была использована группировка по столбцу id таблицы teams, хотя этот столбец не выводится в запросе. Это необходимо для того, чтобы команды с одинаковым названием, если такие найдутся, не группировались между собой. Группировка по названию команды в данном запросе будет неверной, так как есть несколько команд с одинаковым полным названием — мы говорили об этом в начале модуля.

Также, применяя агрегатные функции к соединённым таблицам, обращайте внимание на указание алиасов (или таблиц) при группировке и указании столбцов агрегатных функций. В нашей соединённой таблице есть два столбца с названием id, и если бы мы сформировали запрос без указания таблицы, как указано ниже, то...

SELECT
    t.long_name, /*столбец long_name таблицы t*/
    SUM(m.home_team_goals) + SUM(m.away_team_goals) match_goals /*функция суммирования; столбец home_team_goals таблицы m; функция суммирования; столбец away_team_goals таблицы m; новое название столбца*/
FROM
    sql.matches m /*таблица matches с алиасом m*/
    JOIN sql.teams t ON m.away_team_api_id = t.api_id /*оператор соединения таблиц; таблица teams с алиасом t; условие: away_team_api_id таблицы m равен api_id таблицы t*/
GROUP BY id /*группировка по столбцу id, но не указано, какой таблицы*/

система выдала бы уже знакомую нам ошибку: "... column "id" is ambiguous ...".

Мы можем использовать оператор HAVING для фильтрации сгруппированных данных.

Поставим задачу — вывести таблицу с суммарным количеством забитых голов в матчах по командам и сезонам для команд, в которых суммарное количество голов в матчах сезона больше 100.

Для начала посчитаем общее количество голов в матчах по сезонам.

SELECT
    m.season, /*столбец season таблицы m*/
    SUM(m.home_team_goals) + SUM(m.away_team_goals) total_goals /*функция суммирования; столбец home_team_goals таблицы m; функция суммирования; столбец away_team_goals таблицы m; новое название столбца*/
FROM sql.matches m /*таблица matches с алиасом m*/
GROUP BY m.season /*группировка по столбцу season таблицы m*/

Затем добавим таблицу с командами, группировку по командам и условие фильтрации.



SELECT
    m.season, /*столбец season таблицы m*/
    t.long_name, /*столбец long_name таблицы t*/
    SUM(m.home_team_goals) + SUM(m.away_team_goals) total_goals /*функция суммирования; столбец home_team_goals таблицы m; функция суммирования; столбец away_team_goals таблицы m; новое название столбца*/
FROM sql.matches m /*таблица matches с алиасом m*/
JOIN sql.teams t ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id /*оператор соединения таблиц; таблица teams с алиасом t; условие: home_team_api_id таблицы m равен api_id таблицы t или away_team_api_id таблицы m равен api_id таблицы t*/
GROUP BY m.season, t.id /*группировка по столбцам season таблицы m и id таблицы t*/
HAVING SUM(m.home_team_goals) + SUM(m.away_team_goals) > 100 /*оператор фильтрации сгруппированных данных; функция суммирования; home_team_goals таблицы m; функция суммирования; away_team_goals таблицы m; больше 100*/

Оператор OR в строке JOIN действует следующим образом:

JOIN sql.teams t ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id
Этот оператор соединяет таблицу matches с таблицей teams по двум условиям:

t.api_id = m.home_team_api_id: соединяет строки, где api_id команды совпадает с home_team_api_id в таблице matches.
t.api_id = m.away_team_api_id: соединяет строки, где api_id команды совпадает с away_team_api_id в таблице matches.
Таким образом, для каждой строки в таблице matches будет найдено соответствие в таблице teams, если api_id команды совпадает либо с home_team_api_id, либо с away_team_api_id. Это позволяет получить данные о командах, которые участвовали в матче как домашние, так и гостевые команды.

### Задание 4.3 (External resource)

Напишите запрос, с помощью которого можно вывести список полных названий команд, сыгравших в гостях 150 и более матчей. Отсортируйте список по названию команды.

SELECT
    t.long_name /*столбец long_name таблицы t*/
FROM
    sql.matches m /*таблица matches с алиасом m*/
    JOIN sql.teams t ON m.away_team_api_id = t.api_id 
GROUP BY t.id
HAVING COUNT(m.id) >= 150
ORDER BY t.long_name

## 5. Способы соединения таблиц

### Операторы

✍ В прошлых юнитах для соединения таблиц мы использовали один оператор — JOIN. В действительности же он не один: существует несколько различных видов соединений (join’ов) — давайте посмотрим, какие ↓

#### INNER JOIN

INNER JOIN — это тот же JOIN (слово inner в операторе можно опустить).

Для INNER JOIN работает следующее правило: присоединяются только те строки таблиц, которые удовлетворяют условию соединения. Если в любой из соединяемых таблиц находятся такие строки, которые не удовлетворяют заявленному условию, — они отбрасываются.

Рассмотрим на примере нашего датасета. 

В таблице teams есть данные о 299 различных командах — можем проверить это с помощью запроса.

SELECT 
COUNT(DISTINCT id) /*оператор подсчёта строк; оператор исключения повторяющихся строк; столбец id*/
FROM sql.teams /*таблица teams*/

299

или

SELECT 
COUNT(DISTINCT api_id) /*оператор подсчёта строк; оператор исключения повторяющихся строк; столбец api_id*/
FROM sql.teams

Теперь добавим к teams таблицу с матчами.

SELECT 
COUNT(DISTINCT t.id) /*оператор подсчёта строк; оператор исключения повторяющихся строк; столбец id*/
FROM 
sql.teams t /*таблица teams с алиасом t*/
JOIN sql.matches m ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id /*оператор соединения inner JOIN; таблица teams с алиасом t; условие: home_team_api_id таблицы m равен api_id таблицы t или away_team_api_id таблицы m равен api_id таблицы t*/


292

И в таблице останется уже не 299 команд, а только 292.
Почему?

Дело в том, что таблица sql.matches по какой-то причине не содержит информацию о командах Lierse SK, KVC Westerlo, KAS Eupen, Club Brugge KV, KV Oostende, RSC Anderlecht и Hull City, зато они есть в таблице sql.teams. Возможно, эти команды не участвовали ни в одном матче или записи по этим матчам были удалены.

### LEFT OUTER JOIN и RIGHT OUTER JOIN

Также существуют схожие друг с другом типы соединения — LEFT JOIN и RIGHT JOIN (слово outer в операторе можно опустить).

Для LEFT JOIN работает следующее правило: из левой (относительно оператора) таблицы сохраняются все строки, а из правой добавляются только те, которые соответствуют условию соединения. Если в правой таблице не находится соответствия, то значения строк второй таблицы будут иметь значение NULL.


LEFT JOIN может быть полезен, когда соответствующих записей во второй таблице может не быть, но важно сохранить записи из первой таблицы.

?
Почему соответствий может не быть?

Причины могут быть разные, назовём две основные:


1 Различная бизнес-логика таблиц.

Пример: в интернет-магазине почти всегда можно оформить заказ с промокодом. Вполне вероятно, что информация о промокодах хранится в отдельной таблице, но при этом не для каждого заказа будет существовать промокод, поскольку покупку можно оформить и без него.

2  Разное время обновления таблиц.

Пример: клиент зарегистрировался на сайте интернет-магазина и оформил заказ. Данные о регистрации клиентов могут обновляться в базе данных раз в неделю, а о заказах — раз в сутки.

→ Поставим следующую задачу: вывести полные названия команд, данных по которым нет в таблице matches.


SELECT
    t.long_name,
    m.id
FROM sql.teams t
LEFT JOIN sql.matches m ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id
ORDER BY m.id DESC

Вывод: в таблице teams сохранились все записи, а в таблице matches есть пустые строки.

Теперь, чтобы выбрать такие команды, которые не принимали участия в матчах, достаточно добавить условие where m.id is null (или любое другое поле таблицы matches).

SELECT
    t.long_name
FROM 
    sql.teams t
LEFT JOIN sql.matches m ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id
WHERE m.id IS NULL

Обратите внимание! Если мы добавим какой-либо фильтр по отличному от NULL значению для таблицы matches, то LEFT JOIN превратится в INNER JOIN, поскольку для второй таблицы станет необходимым присутствие такого (NOT NULL) значения в строке.

SELECT
    t.long_name FROM sql.teams t
LEFT JOIN sql.matches m ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id
WHERE
    m.season = '2008/2009'
AND t.long_name = 'KAS Eupen'

В ответе Metabase получим No results!, так как вместе с фильтром m.season = '2008/2009' исчезли все строки, значения которых NULL.

### Задание 5.1 (External resource)

Используя LEFT JOIN, выведите список уникальных названий команд, содержащихся в таблице matches. Отсортируйте список в алфавитном порядке.

SELECT DISTINCT
    t.long_name
FROM 
    sql.teams t
LEFT JOIN sql.matches m ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id
WHERE m.id IS NOT NULL
ORDER BY t.long_name ASC


Если убрать строку WHERE m.id IS NOT NULL, то запрос будет возвращать все команды, включая те, которые не имеют соответствующих записей в таблице matches. Это произойдет потому, что левое соединение (LEFT JOIN) вернет все строки из таблицы teams, а для тех строк, которые не имеют соответствующих записей в таблице matches, значения столбцов из таблицы matches будут NULL. Если вам нужны только те команды, которые участвовали в матчах, строку WHERE m.id IS NOT NULL необходимо оставить.


С LEFT JOIN также работают агрегатные функции, что позволяет не потерять значения из левой таблицы. Например, мы можем вывести сумму голов команд по гостевым матчам.

SELECT
    t.long_name,
    SUM(m.away_team_goals) total_goals
FROM 
    sql.teams t /*таблица teams с алиасом t*/
LEFT JOIN sql.matches m ON t.api_id = m.away_team_api_id /*оператор соединения LEFT JOIN; таблица matches с алиасом m; условие: away_team_api_id таблицы m равен api_id таблицы t*/
GROUP BY t.id /*группировка по столбцу id таблицы t*/
ORDER BY 2 DESC /*сортировка по столбцу total_goals по убыванию, чтобы увидеть строки со значением null*/

Обратите внимание! При применении функций SUM, MIN, MAX, AVG к полям со значением NULL в результате получится NULL, а не 0. А при использовании функции COUNT, наоборот, получится 0.

### Задание 5.2 (External resource)

Используя LEFT JOIN, напишите запрос, который выведет полное название команды (long_name), количество матчей, в которых участвовала команда, — домашних и гостевых (matches_cnt). Отсортируйте по количеству матчей в порядке возрастания, затем — по названию команды в алфавитном порядке.

SELECT
    t.long_name,
    COUNT(m.id) matches_cnt
FROM 
    sql.teams t 
LEFT JOIN sql.matches m ON t.api_id = m.away_team_api_id OR t.api_id = m.home_team_api_id 
GROUP BY t.id, t.long_name
ORDER BY matches_cnt ASC, t.long_name ASC 

SELECT 
    t.long_name,
    COUNT(m.id) AS matches_cnt
FROM 
    sql.teams t
LEFT JOIN 
    sql.matches m ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id
GROUP BY 
    t.id, t.long_name
ORDER BY 
    matches_cnt ASC, t.long_name ASC;

### При использовании RIGHT JOIN сохраняется та же логика, что и для LEFT JOIN, только за основу берётся правая таблица.

Чтобы из LEFT JOIN получить RIGHT JOIN, нужно просто поменять порядок соединения таблиц.

**Вообще, применение RIGHT JOIN считается плохим тоном, так как язык SQL читается и пишется слева направо, а такой оператор усложняет чтение запросов.**


### FULL OUTER JOIN

Оператор FULL OUTER JOIN объединяет в себе LEFT и RIGHT JOIN и позволяет сохранить кортежи обеих таблиц. Даже если не будет соответствий, мы сохраним все записи из обеих таблиц.

FULL OUTER JOIN может быть полезен в ситуациях, когда схема данных недостаточно нормализована и не хватает таблиц-справочников.

Пример: в базе данных интернет-магазина есть две таблицы — с зарегистрированными пользователями и пользователями, оформившими заказ. При этом оформить заказ можно без регистрации, а зарегистрироваться — без оформления заказа.

Предположим, что вам необходимо получить полный список пользователей — и оформивших заказ, и зарегистрированных, — но в базе данных этой объединённой таблицы нет. В данном случае можно использовать FULL OUTER JOIN для получения полного списка, соединив таким образом таблицы c заказами и регистрациями по id пользователя.

Синтаксис FULL OUTER JOIN аналогичен другим JOIN.

SELECT 
…
FROM
	table1
FULL OUTER JOIN table2 ON условие

### Cross Join



На самом деле с этим оператором соединения таблиц вы познакомились в самом начале текущего модуля.

CROSS JOIN соединяет таблицы так, что каждая запись в первой таблице присоединяется к каждой записи во второй таблице, иначе говоря, даёт декартово произведение.

В начале модуля вы использовали этот оператор, записывая таблицы через запятую.

SELECT *
FROM
    sql.teams,
    sql.matches

Этот же запрос можно записать с использованием CROSS JOIN.

SELECT *
FROM
    sql.teams
    CROSS JOIN sql.matches

Обратите внимание! Условие для CROSS JOIN, в отличие от других операторов, не требуется.

Также этот запрос можно записать с помощью INNER JOIN с условием on true — результат будет тот же.

SELECT *
FROM
    sql.teams
    JOIN sql.matches ON TRUE

CROSS JOIN может быть полезен, когда необходимо создать таблицу фактов.

Например, с помощью такого запроса мы можем получить все возможные комбинации полных названий команд в матчах.

SELECT
    DISTINCT /*оператор исключения повторяющихся строк*/
    t1.long_name home_team, /*столбец long_name таблицы t1; новое название*/
    t2.long_name away_team /*столбец long_name таблицы t2; новое название*/
FROM
    sql.teams t1 /*таблица teams с алиасом t1*/
    CROSS JOIN sql.teams t2 /*оператор соединения CROSS JOIN; таблица teams с алиасом t2*/

### Задание 5.3 (External resource)

Напишите запрос, который выведет все возможные уникальные комбинации коротких названий домашней команды (home_team) и коротких названий гостевой команды (away_team). Команда не может сама с собой играть, то есть быть и домашней, и одновременно гостевой (в одном и том же матче). Отсортируйте запрос по первому и второму столбцам.

SELECT
    DISTINCT /*оператор исключения повторяющихся строк*/
    t1.short_name home_team, /*столбец short_name таблицы t1; новое название*/
    t2.short_name away_team /*столбец short_name таблицы t2; новое название*/
FROM
    sql.teams t1 /*таблица teams с алиасом t1*/
    CROSS JOIN sql.teams t2 /*оператор соединения CROSS JOIN; таблица teams с алиасом t2*/
WHERE t1.short_name != t2.short_name
ORDER BY
    home_team ASC, away_team ASC;

SELECT DISTINCT
    ht.short_name AS home_team,
    at.short_name AS away_team
FROM
    sql.teams ht
CROSS JOIN
    sql.teams at
WHERE
    ht.id <> at.id
ORDER BY
    home_team ASC, away_team ASC;

### NATURAL JOIN

Ключевое слово natural в начале оператора JOIN позволяет не указывать условие соединения таблиц — для соединения будут использованы столбцы с одинаковым названием из этих таблиц.

NATURAL JOIN можно использовать с любыми видами соединений, которые требуют условия соединения:

→ NATURAL INNER JOIN (возможна запись NATURAL JOIN);
→ NATURAL LEFT JOIN;
→ NATURAL RIGHT JOIN;
→ NATURAL FULL OUTER JOIN.


При использовании NATURAL JOIN прежде всего стоит обратить внимание на ключи таблиц. Для наших таблиц teams и matches этот вид соединения не подойдёт, так как общим для обеих таблиц является столбец id, но таблицы соединяются по другим столбцам.

Когда у таблиц есть несколько столбцов с одинаковыми именами, при NATURAL JOIN условие соединения будет применено на все столбцы с одинаковыми именами.

То есть для таблиц table1 и table2

table1: id, name, ...

table2: id, name, ...

запрос

SELECT 
…
FROM          table1 NATURAL JOIN table2


будет равнозначен запросу

SELECT
…
FROM          table1 t1
INNER JOIN table2 t2 ON t1.id = t2.id AND t1.name = t2.name

### Общая логика построения запроса с JOIN

При построении запроса с несколькими JOIN старайтесь идти слева направо. Сначала выберите таблицу, которая является центральной в соответствии с поставленной задачей, вопросом. Затем добавляйте таблицы поэтапно в зависимости от бизнес-логики запроса.

Например, для ответа на вопрос: «Какая команда сыграла больше всех матчей в сезоне 2010/2011?» в качестве центральной лучше выбрать таблицу с командами.

А для ответа на вопрос: «В каком сезоне участвовало больше всего команд?» — таблицу с матчами.

Стоит отметить, что из рассмотренных видов соединений чаще всего используются INNER JOIN и LEFT JOIN. Другие операторы используются реже, но стоит помнить об их существовании при решении нестандартных задач.

## 6. Итоги. Закрепление знаний

✍ В этом модуле вы познакомились с разными способами вертикального соединения таблиц.

Теперь вы умеете:

соединять таблицы по ключу;
соединять таблицы с помощью оператора JOIN;
использовать разные виды JOIN;
применять агрегации к объединённым таблицам и фильтровать вывод.
Теперь давайте закрепим полученные знания ↓

Вам предстоит выполнить пять заданий на написание SQL-запросов по определённым условиям.

Если какую-то задачу не получается выполнить с первого раза — не отчаивайтесь. Соединение таблиц — одна из самых сложных тем для освоения и эффективного использования языка SQL.

Если при выполнении заданий возникли трудности, попробуйте вернуться к материалам и посмотреть примеры запросов для разных задач.

Также вы всегда можете спросить совета у менторов или сокурсников в Пачке.

### Задание 6.1 (External resource)

Напишите запрос, который выведет список уникальных полных названий команд (long_name), игравших в гостях в матчах сезона 2012/2013. Отсортируйте список в алфавитном порядке.

SELECT DISTINCT /*оператор исключения повторяющихся строк*/
    t.long_name
FROM
    sql.teams t 
LEFT JOIN sql.matches m ON t.api_id = m.away_team_api_id
WHERE m.season = '2012/2013' 
GROUP BY t.id, t.long_name
ORDER BY t.long_name ASC 


### Задание 6.2 (External resource)

Напишите запрос, который выведет полное название команды (long_name) и общее количество матчей (matches_cnt), сыгранных командой Inter в домашних матчах.

SELECT 
    t.long_name,
    count(m.id) as matches_cnt
FROM
    sql.teams t 
LEFT JOIN sql.matches m ON t.api_id = m.home_team_api_id
WHERE t.long_name = 'Inter' 
GROUP BY t.id, t.long_name


### Задание 6.3 (External resource)

Напишите запрос, который выведет топ-10 команд (long_name) по суммарному количеству забитых голов в гостевых матчах. Во втором столбце запроса выведите суммарное количество голов в гостевых матчах (total_goals).

SELECT 
    t.long_name,
    SUM(m.away_team_goals) as total_goals
FROM
    sql.teams t 
LEFT JOIN sql.matches m ON t.api_id = m.away_team_api_id
GROUP BY t.id, t.long_name
ORDER BY total_goals DESC NULLS LAST
LIMIT 10

### Задание 6.4 (External resource)

Выведите количество матчей между командами Real Madrid CF и FC Barcelona. В поле ниже введите запрос, с помощью которого вы решили задание.

SELECT COUNT(*) AS match_count
FROM sql.matches m
JOIN sql.teams ht ON m.home_team_api_id = ht.api_id
JOIN sql.teams at ON m.away_team_api_id = at.api_id
WHERE ht.long_name = 'Real Madrid CF'
  AND at.long_name = 'FC Barcelona'
   OR ht.long_name = 'FC Barcelona'
  AND at.long_name = 'Real Madrid CF';

Этот запрос объединяет таблицы matches и teams дважды: один раз для домашней команды и один раз для гостевой команды. Затем он фильтрует результаты, чтобы найти матчи, где одна команда - Real Madrid CF, а другая - FC Barcelona, независимо от того, какая из них была домашней или гостевой.

### Задание 6.5 (External resource)

Напишите запрос, который выведет название команды (long_name), сезон (season) и суммарное количество забитых голов в домашних матчах (total_goals). Оставьте только те строки, в которых суммарное количество голов менее десяти. Отсортируйте запрос по названию команды, а затем — по сезону.

SELECT 
    ht.long_name,
    m.season,
    sum(m.home_team_goals) AS total_goals
FROM sql.matches m
    JOIN sql.teams ht ON m.home_team_api_id = ht.api_id
group by ht.id, m.season
HAVING SUM(m.home_team_goals) < 10
ORDER BY ht.long_name ASC, m.season ASC


Отлично, вы справились!

Однако мы предлагаем немного задержаться в этом модуле.

Объединение таблиц — одна из ключевых тем. Знания в этой области нередко проверяют на собеседованиях, поэтому уделите отработке навыков особое внимание. Ниже вы найдёте несколько наиболее распространённых вопросов по объединению таблиц.

Важно! Это задания на самопроверку, они необязательные и мы не будем оценивать качество их решения. Однако настоятельно рекомендуем вам попробовать выполнить их самостоятельно, прежде чем открывать ответ.

Вопрос 1
Задание от Лудамилы Джанда (Ludamila Janda),
Data Scientist в Amplify

Объясните разницу между левым (LEFT) и внутренним (INNER) соединениями.

ПРИМЕР ОТВЕТА
Соединения — это способы объединения данных из двух разных таблиц (левой и правой) в новую.

Принцип работы соединений заключается в объединении строк между двумя таблицами; набор ключевых столбцов используется для поиска данных в двух одинаковых таблицах, которые необходимо соединить.

В случае левого соединения каждая строка из левой таблицы появляется в полученной таблице, но строки из правой появляются, только если значения в их ключевых столбцах отображаются в левой. При внутреннем соединении обе строки из левой и правой таблиц появляются, только если есть совпадающая строка в другой таблице.

На практике вы можете воспринимать левое соединение как добавление данных из правой таблицы к левой, если она существует (например, с использованием правой таблицы в качестве подстановочной). Внутреннее соединение больше похоже на поиск всех общих данных и создание новой таблицы только из пар.

Обратите внимание, что в нашем варианте ответа мы не упоминали какие‐либо сложности, связанные с появлением повторяющихся строк в данных. Возможно, о них стоит упомянуть, потому что повторяющиеся строки могут повлиять на результат, но с большей долей вероятности эта информация может увести вас от мысли, которую вы пытаетесь донести до интервьюера.

Лудамила Джанда любит задавать этот вопрос при наборе кандидатов на роли джуниоров, потому что это не вопрос с подвохом, а минимально необходимые знания. Она считает, что можно многое узнать по тому, как кандидат решает дать ответ. Правильно ответить можно по‐разному: процитировать учебник, используя сложные выражения, или объяснить всё простым языком, не вдаваясь в крайности.

Вопрос 2
Перед вами школьная таблица TABLE_A с оценками от 0 до 100, выставленными ученикам нескольких классов. 



Как бы вы рассчитали максимальную оценку в каждом классе?

ПРИМЕР ОТВЕТА
Запрос для поиска наивысшей оценки в каждом классе может выглядеть так:

SELECT 
  CLASS, 
  MAX(GRADE) INTO TABLE_B 
FROM 
  TABLE_A 
GROUP BY 
  CLASS
Этот запрос группирует данные по каждому классу, а затем находит максимальное значение. Он дополнительно сохраняет результаты в новую таблицу (TABLE_B), чтобы их можно было запросить позже.

У соискателей этот вопрос обычно вызывает трудности. Одна из причин — непонимание, что именно нужно группировать (в данном случае переменную класса). Другая — в том, что вопрос кажется настолько простым, что соискатели начинают слишком усложнять ответ и упускают простое решение.

Если вы услышали вопрос, который кажется вам слишком лёгким, вполне возможно, что таким он и является.

К слову, строка INTO TABLE_B была совершенно необязательной, но она хорошо подготовит вас к следующему вопросу.

Это один из простейших вопросов по SQL: он проверяет, имеете ли вы базовое представление о группировке в SQL. Если решение не кажется вам очевидным, то самое время повторить, как группирующие переменные работают в SQL.

Вопрос 3
Рассмотрим таблицу из предыдущего вопроса.



Как найти не только самую высокую оценку в каждом классе и учащегося, получившего её?

ПРИМЕР ОТВЕТА
Предполагая, что у нас есть результат предыдущего запроса, сохранённый в TABLE_B, мы можем использовать его в этом решении:

SELECT 
  a.CLASS, 
  a.GRADE, 
  a.STUDENT 
FROM 
  TABLE_A a 
  INNER JOIN TABLE_B b ON a.CLASS = b.CLASS 
  AND a.GRADE = b.GRADE
Этот запрос выбирает всех учащихся и их оценки из исходной таблицы TABLE_A, где указаны классы и оценки, которые отображаются в таблице максимальных значений TABLE_B. Внутреннее соединение работает как фильтр, сохраняя только максимальные комбинации классов/оценок, потому что только в этом случае оценка появляется в таблице TABLE_B.

В качестве альтернативы можно было бы использовать подзапрос, чтобы сделать то же самое, не вызывая таблицу TABLE_B:

SELECT 
  a.CLASS, 
  a.GRADE, 
  a.STUDENT 
FROM 
  TABLE_A a 
  INNER JOIN (
    SELECT 
      CLASS, 
      MAX(GRADE) 
    FROM 
      TABLE_A 
    GROUP BY 
      CLASS
  ) b ON a.CLASS = b.CLASS 
  AND a.GRADE = b.GRADE
Примечание. Подзапросы мы будем проходить чуть позднее, так что, ознакомившись сейчас, вернитесь к этому варианту, когда изучите синтаксис подзапросов.

Как видите, эту задачу можно решить несколькими способами, но любое решение почти наверняка требует более одного запроса из таблицы TABLE_A, поэтому этот вопрос может легко поставить в тупик. На бумаге решение может показаться лёгким, но додуматься до него во время интервью может быть непросто. Даже если вы ответили неправильно, это не значит, что вы провалились.

Решение не даёт каких‐либо специфичных случаев для привязки к максимальному значению. В примере будут возвращены строки для нескольких студентов. Возможно, стоит указать на этот факт интервьюеру, потому что так вы показываете, что обращаете внимание на пограничные случаи.